In [28]:
# default_exp data.finance

In [29]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [30]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:
from libs import is_lib_exists

In [32]:
required_libs = [ ("numpy", "numpy"),
                  ("pandas", "pandas"),
                  ("seaborn", "seaborn"),
                  ("matplotlib", "matplotlib"),
                  ("investpy", "investpy"),
                  ("dateutil", "python-dateutil"),
                ]

In [33]:
for (clz,lib) in required_libs:
    if not is_lib_exists(clz):
        print(f"Installing {lib}")
        !pip install {lib}
    else:
        print(f"{lib} exists")

numpy exists
pandas exists
seaborn exists
matplotlib exists
investpy exists
python-dateutil exists


In [34]:
#export
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import investpy
from pathlib import Path
from dateutil.relativedelta import *
from datetime import *
import calendar
import traceback

In [35]:
%matplotlib inline

In [36]:
# % - line, %% - cell
# %run, %%timeit, %%writefile, %pycat
# %who, %store
# %html 
# %env, %pinfo
# %lsmagic
# %history
# %prun
# %%heat
# https://github.com/lmmx/devnotes/wiki/IPython-'magic'-function-documentation#edit
# %quickref

# https://ipython.readthedocs.io/en/stable/interactive/magics.html#

In [37]:
#%%bash
#for i in {2..10}
#do
#echo $i
#done

In [38]:
from IPython.display import display, Image

# Finance Data

> Functionality to gather finance data.

The classes here provide bases classes used to gather finance data for model training.

In [39]:
#hide
from nbdev.showdoc import *

In [40]:
# Dataset path
DATASET_PATH = Path("../../data")  

# Country to analyze
COUNTRY = "malaysia"

# Dataset
DATASET = DATASET_PATH/f"ftse_klse_investing.csv"

# Stocks summary
DATASET_STOCKS_SUMMARY = DATASET_PATH/f"stocks_summary.csv"

## FTSE KLSE

In [41]:
# Retrieve stock components
dataset = investpy.get_stocks(country=COUNTRY)
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 929 entries, 0 to 928
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   country    929 non-null    object
 1   name       929 non-null    object
 2   full_name  929 non-null    object
 3   isin       929 non-null    object
 4   currency   929 non-null    object
 5   symbol     929 non-null    object
dtypes: object(6)
memory usage: 43.7+ KB


In [44]:
# Save the dataset
dataset.to_csv(DATASET, index=False)

#dataset = pd.read_csv(DATASET, header=0, sep=',', quotechar='"')

In [45]:
display(dataset.head(3))

,country,name,full_name,isin,currency,symbol
0,malaysia,Media Chinese Int,Media Chinese International Ltd,BMG5959D1048,MYR,MDCH
1,malaysia,AMMB,AMMB Holdings Bhd,MYL1015OO006,MYR,AMMB
2,malaysia,CIMB Group,CIMB Group Holdings Bhd,MYL1023OO000,MYR,CIMB


## TODO - stock sector, balance sheet and P&L

In [46]:
def calculate_yield(idx, name, symbol):
    """Calculate stock yield based on latest close price"""
    try:
        df_dividends = investpy.get_stock_dividends(stock=symbol, country=COUNTRY)
        #display(df_dividends)
        df_dividends['Year'] = pd.DatetimeIndex(df_dividends['Date']).year
        # Convert yield percentage to number
        df_dividends['Historical Yield'] =  df_dividends['Yield'].apply(\
                                                     lambda y : \
                                                     float(y.translate(str.maketrans({',': '.', '%': '', '-':'0'}))))
        
        # Aggregate the dividends by years
        df_dividends_summary = df_dividends.groupby(by=['Year'], sort=False, as_index=False).\
                                agg({'Dividend':'sum', 'Historical Yield':'sum', \
                                     'Date': lambda d: ','.join(d.dt.strftime('%Y-%m-%d'))})
        # print(df_dividends.groupby(by=['Year'], sort=False, as_index=False))
        # display(df_dividends_summary[df_dividends_summary['Year'].isin(PAST_3_YEARS)])
        #display(df_dividends_summary)
        
        df_recent_data = investpy.get_stock_recent_data(stock=symbol, country=COUNTRY).reset_index()
        last_close_price = df_recent_data[df_recent_data['Date'].eq(df_recent_data['Date'].max())]['Close'].values[0]
        # display(f"Recent price {last_close_price}")
        # display(df_recent_data)
     
        df_dividends_summary['Name'] = name
        df_dividends_summary['Symbol'] = symbol
        df_dividends_summary['Last Close Price'] = last_close_price
        df_dividends_summary['Calculated Yield'] = df_dividends_summary['Dividend'] / last_close_price * 100
       
        # display(df_dividends_summary)
        
        
        print(f'{idx}. {symbol} - {name}')
        return df_dividends_summary
            
    except Exception as exception:
        print(f"{idx}. Skipping {name} - {symbol}")
        #traceback.print_exc()
        return pd.DataFrame()

In [47]:
def summarize_stocks(df_summary, df):
    """Summarize the stock dividends details"""
    if df is None or df.empty:
        return df_summary
    if df_summary.empty:
        return df
    return df_summary.append(df)

In [48]:
import time

def generate_dataset():
    #!rm -rf {DATASET_STOCK_SUMMARY}
    df_all_dividends = pd.DataFrame()

    #df = calculate_yield(0, "BIMB", "BIMB")
    #df_all_dividends = summarize_stocks(df_all_dividends, df)
    #display(df_all_dividends)

    START_INDEX = 0
    for idx, row in dataset.iloc[START_INDEX:].iterrows():
        name = row['name']
        symbol = row['symbol']
        df = calculate_yield(idx, name, symbol)
        df_all_dividends = summarize_stocks(df_all_dividends, df)
        if (idx % 10 == 0 or idx == (len(dataset) - 1)):
            if (idx >= 15) 
                header = False
            else
                header = True
            df_all_dividends.to_csv(DATASET_STOCKS_SUMMARY, mode='a', index=False, header=header)
            df_all_dividends = pd.DataFrame()
        time.sleep(3)
    # display(df_all_dividends)

# Generate the dataset again?
#generate_dataset()

0. MDCH - Media Chinese Int
1. AMMB - AMMB
2. CIMB - CIMB Group
3. RHBC - RHB Bank
4. HLCB - Hong Leong Financial
5. MBBM - Malayan Banking
6. MBSS - Malaysia Building
7. PUBM - Public Bank
8. BSTB - Berjaya Sports Toto
9. DRBM - DRB Hicom
10. MYRS - Malaysian Resources
11. BMYS - Bursa Malaysia
12. IOIB - IOI Corp
13. TDMS - TDM
14. MMCB - MMC Corp
15. GENP - Genting Plantations
16. KLKK - Kuala Lumpur Kepong
17. ALLI - Alliance Bank Malaysia
18. BOUS - Boustead
19. CBMS - Carlsberg Brewery
20. DBMS - Dutch Lady Milk
21. HAPS - Hap Seng Consolidated
22. GENT - Genting
23. IJMS - IJM Corp
24. BGRO - Berjaya Corp
25. ENOB - Eastern & Oriental
26. MALY - Lafarge Malayan Cement
27. MISC - MISC
28. MAGM - Magnum
29. Skipping Mulpha Int - MITC
30. PEPT - PPB
31. BATO - BAT Malaysia
32. SIME - Sime Darby
33. MPRM - Media Prima
34. UMWS - UMW
35. YTLS - YTL Corp
36. TLMM - Telekom Malaysia
37. TCOM - TIME Dotcom
38. THPB - TH Plantations
39. ZHCO - Zhulian Corp
40. UMSB - UEM Sunrise
41. TAGL

In [60]:
dataset = pd.read_csv(DATASET_STOCKS_SUMMARY, header=0, sep=',', quotechar='"')
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2633 entries, 0 to 2632
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Year              2633 non-null   int64  
 1   Dividend          2633 non-null   float64
 2   Historical Yield  2633 non-null   float64
 3   Date              2633 non-null   object 
 4   Name              2633 non-null   object 
 5   Symbol            2633 non-null   object 
 6   Last Close Price  2633 non-null   float64
 7   Calculated Yield  2633 non-null   float64
dtypes: float64(4), int64(1), object(3)
memory usage: 164.7+ KB


In [61]:
# Today's date
TODAY = date.today()

# Past 3 years
#PAST_3_YEARS = [ TODAY.year - y for y in range(1,4) ]
PAST_3_YEARS = [ TODAY.year - y for y in range(0,3) ]
print(PAST_3_YEARS)

# Expected YIELD %
EXPECTED_YIELD_PERCENTAGE = 15

[2020, 2019, 2018]


In [62]:
df_dividend_stocks = dataset[
                 (dataset['Year'].isin(PAST_3_YEARS)) & 
                 (dataset['Calculated Yield'] >= EXPECTED_YIELD_PERCENTAGE)]

#df_dividend_stocks = df_dividend_stocks.sort_values(by=['Name', 'Year'])

In [63]:
for symbol in df_dividend_stocks['Symbol'].unique():
    display(dataset[dataset['Symbol']==symbol])

,Year,Dividend,Historical Yield,Date,Name,Symbol,Last Close Price,Calculated Yield
0,2020,0.0041,2.48,2020-07-10,Media Chinese Int,MDCH,0.16,2.5625
1,2019,0.0226,12.90,"2019-12-09,2019-06-19",Media Chinese Int,MDCH,0.16,14.1250
2,2018,0.0322,16.50,"2018-12-12,2018-06-18",Media Chinese Int,MDCH,0.16,20.1250
3,2017,0.0557,15.78,"2017-12-11,2017-06-16",Media Chinese Int,MDCH,0.16,34.8125
4,2016,0.0879,13.60,"2016-12-13,2016-06-20",Media Chinese Int,MDCH,0.16,54.9375
5,2015,0.0790,6.18,"2015-12-08,2015-07-08",Media Chinese Int,MDCH,0.16,49.3750


,Year,Dividend,Historical Yield,Date,Name,Symbol,Last Close Price,Calculated Yield
48,2020,0.3222,14.92,"2020-12-30,2020-03-10",Berjaya Sports Toto,BSTB,2.21,14.579186
49,2019,0.6350,25.35,"2019-12-12,2019-07-18,2019-04-10,2019-01-16",Berjaya Sports Toto,BSTB,2.21,28.733032
50,2018,0.6100,26.95,"2018-10-03,2018-07-16,2018-04-11,2018-01-24",Berjaya Sports Toto,BSTB,2.21,27.601810
51,2017,0.6100,24.15,"2017-10-06,2017-07-18,2017-04-11,2017-01-05",Berjaya Sports Toto,BSTB,2.21,27.601810
52,2016,0.5189,16.22,"2016-07-18,2016-04-11,2016-01-18",Berjaya Sports Toto,BSTB,2.21,23.479638
53,2015,0.1839,5.99,2015-10-08,Berjaya Sports Toto,BSTB,2.21,8.321267


,Year,Dividend,Historical Yield,Date,Name,Symbol,Last Close Price,Calculated Yield
105,2018,0.3600,23.35,"2018-12-13,2018-09-19,2018-06-13,2018-03-13",Boustead,BOUS,0.615,58.536585
106,2017,0.5707,20.66,"2017-12-14,2017-09-14,2017-06-13,2017-03-14",Boustead,BOUS,0.615,92.796748
107,2016,0.6173,24.09,"2016-12-13,2016-09-09,2016-06-08,2016-03-29",Boustead,BOUS,0.615,100.373984
108,2015,0.4200,5.22,"2015-12-22,2015-09-15",Boustead,BOUS,0.615,68.292683


,Year,Dividend,Historical Yield,Date,Name,Symbol,Last Close Price,Calculated Yield
109,2020,3.6309,12.95,"2020-09-04,2020-05-15,2020-03-26,2020-01-02",Carlsberg Brewery,CBMS,23.22,15.636951
110,2019,4.3520,17.91,"2019-10-18,2019-07-17,2019-05-16,2019-03-27,20...",Carlsberg Brewery,CBMS,23.22,18.742463
111,2018,1.9770,10.25,"2018-09-18,2018-06-18,2018-05-02",Carlsberg Brewery,CBMS,23.22,8.514212
112,2017,1.4900,9.56,"2017-09-19,2017-05-03",Carlsberg Brewery,CBMS,23.22,6.416882
113,2016,1.4400,9.72,"2016-09-21,2016-05-03",Carlsberg Brewery,CBMS,23.22,6.201550
114,2015,0.7100,5.83,2015-09-22,Carlsberg Brewery,CBMS,23.22,3.057709


,Year,Dividend,Historical Yield,Date,Name,Symbol,Last Close Price,Calculated Yield
147,2020,1.200,16.24,"2020-12-02,2020-08-27,2020-05-22,2020-03-02",MISC,MISC,6.72,17.857143
148,2019,1.200,16.18,"2019-11-26,2019-08-27,2019-06-10,2019-03-06",MISC,MISC,6.72,17.857143
149,2018,1.200,17.72,"2018-11-30,2018-08-17,2018-05-24,2018-02-26",MISC,MISC,6.72,17.857143
150,2017,0.720,9.95,"2017-11-15,2017-08-21,2017-05-19",MISC,MISC,6.72,10.714286
151,2016,0.825,10.65,"2016-08-18,2016-04-18,2016-02-22",MISC,MISC,6.72,12.276786


...

,Year,Dividend,Historical Yield,Date,Name,Symbol,Last Close Price,Calculated Yield
2582,2020,0.2640,21.65,"2020-12-23,2020-08-05,2020-02-05",UOA REIT,UOAR,1.22,21.639344
2583,2019,0.1846,14.94,"2019-08-06,2019-02-04",UOA REIT,UOAR,1.22,15.131148
2584,2018,0.0829,5.96,2018-08-09,UOA REIT,UOAR,1.22,6.795082


,Year,Dividend,Historical Yield,Date,Name,Symbol,Last Close Price,Calculated Yield
2599,2020,0.2919,27.44,"2020-11-12,2020-08-21,2020-06-12,2020-02-20",Atrium,ATRL,1.16,25.163793
2600,2019,0.2930,26.69,"2019-11-13,2019-07-17,2019-03-21,2019-02-12",Atrium,ATRL,1.16,25.258621
2601,2018,0.1525,13.80,"2018-08-10,2018-05-14",Atrium,ATRL,1.16,13.146552


,Year,Dividend,Historical Yield,Date,Name,Symbol,Last Close Price,Calculated Yield
2602,2020,0.0615,14.73,"2020-12-08,2020-06-04",AmFIRST,AMFL,0.415,14.819277
2603,2019,0.0786,17.02,"2019-12-03,2019-05-09",AmFIRST,AMFL,0.415,18.939759
2604,2018,0.0815,14.57,"2018-12-03,2018-05-07",AmFIRST,AMFL,0.415,19.638554


,Year,Dividend,Historical Yield,Date,Name,Symbol,Last Close Price,Calculated Yield
2605,2020,0.1310,20.07,"2020-09-07,2020-03-06",AmanahRaya Real Estate,AMRY,0.66,19.848485
2606,2019,0.2400,30.14,"2019-12-11,2019-09-18,2019-06-14,2019-03-13",AmanahRaya Real Estate,AMRY,0.66,36.363636
2607,2018,0.1745,20.79,"2018-12-13,2018-09-14,2018-06-19",AmanahRaya Real Estate,AMRY,0.66,26.439394


,Year,Dividend,Historical Yield,Date,Name,Symbol,Last Close Price,Calculated Yield
2608,2020,0.0761,11.05,"2020-08-10,2020-02-05",Tower Real Estate,TWRE,0.58,13.120690
2609,2019,0.0593,7.32,2019-08-09,Tower Real Estate,TWRE,0.58,10.224138
2610,2018,0.1500,16.05,"2018-11-09,2018-08-13,2018-02-19",Tower Real Estate,TWRE,0.58,25.862069


In [66]:
investpy.get_stock_recent_data("PGAS", COUNTRY)

,Open,High,Low,Close,Volume,Currency
Date,,,,,,
2020-11-30,16.98,18.16,16.62,18.16,3359800,MYR
2020-12-01,17.56,17.80,17.10,17.80,718800,MYR
2020-12-02,17.50,17.80,17.48,17.80,957300,MYR
2020-12-03,17.86,17.96,17.48,17.60,947100,MYR
2020-12-04,17.58,17.58,17.02,17.40,222500,MYR
2020-12-07,17.30,17.68,17.12,17.20,1061000,MYR
2020-12-08,17.20,17.62,17.18,17.62,540200,MYR
2020-12-09,17.62,17.62,17.14,17.34,684300,MYR
2020-12-10,17.40,17.40,17.20,17.26,244800,MYR


## Testing

In [64]:
investpy.get_index_recent_data(index="KL Construction", country=COUNTRY)

,Open,High,Low,Close,Volume,Currency
Date,,,,,,
2020-11-30,174.52,175.01,172.24,172.24,229508496,MYR
2020-12-01,172.04,176.63,172.03,176.63,313066688,MYR
2020-12-02,176.89,177.30,175.39,177.30,283734784,MYR
2020-12-03,177.76,180.87,177.64,180.87,1041849408,MYR
2020-12-04,181.33,181.55,179.46,181.47,438255808,MYR
2020-12-07,182.45,182.48,181.34,181.88,1009920000,MYR
2020-12-08,183.02,183.32,179.58,179.68,441040096,MYR
2020-12-09,180.92,181.62,180.04,180.51,370906912,MYR
2020-12-10,181.05,181.81,180.49,181.24,232006000,MYR


In [25]:
display(investpy.get_stock_information("BATO", country=COUNTRY))

,Stock Symbol,Prev. Close,Todays Range,Revenue,Open,52 wk Range,EPS,Volume,Market Cap,Dividend (Yield),Average Vol. (3m),P/E Ratio,Beta,1-Year Change,Shares Outstanding,Next Earnings Date
0,BATO,14.16,13.92 - 14.26,2.320000e+09,14.18,9.26 - 15.64,0.93,366400.0,4.000000e+09,0.89 (6.29%),968803.0,15.15,0.49,- 7.45%,285530000.0,17/02/2021


In [26]:
investpy.get_indices(COUNTRY)

,country,name,full_name,symbol,currency,class,market
0,malaysia,MSCI Malaysia Net USD,MSCI Malaysia Net USD,MIMY00000NUS,USD,major_indices,global_indices
1,malaysia,MSCI Malaysia,MSCI Malaysia,MIMY00000PMY,MYR,major_indices,global_indices
2,malaysia,DJ Malaysia USD,Dow Jones Malaysia USD,MYDOWD,USD,major_indices,global_indices
3,malaysia,KL Construction,KL Construction,KLCT,MYR,primary_sectors,world_indices
4,malaysia,DJ Islamic Malaysia Titans 25,Dow Jones Islamic Market Malaysia Titans 25,DJMY25,MYR,other_indices,global_indices
5,malaysia,DJ Islam Malaysia Titans 25 USD,Dow Jones Islamic Market Malaysia Titans 25 USD,DJMY25D,USD,other_indices,global_indices
6,malaysia,FTSE4Good Bursa Malaysia,FTSE4Good Bursa Malaysia,FTF4GBM,MYR,other_indices,global_indices
7,malaysia,FTSE Bursa Malaysia KLCI,FTSE Bursa Malaysia KLCI,FTFBMKLCI,MYR,other_indices,global_indices
8,malaysia,KL Energy,KL Energy,KLEN,MYR,primary_sectors,world_indices
9,malaysia,MSCI Malaysia Gross,MSCI International Malaysia Gross Real time,MIMY00000GUS,USD,other_indices,global_indices
